In [1]:
%load_ext autoreload
%autoreload 2

import nest
import yaml
from pathlib import Path
from pprint import pprint

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from nets import *

In [4]:
PARAMS_DIR = Path('./params')
OUTPUT_DIR = Path('./output')

# Let's familiarize ourselves with the ``Network`` object

__...by building the elements of the network one by one__

In this section we:

1. Initialize an empty ``Network`` object
2. __Initialize all the elements of the network__ using  the ``Network.build_*`` methods
    1. Models (neuron models, recorder models, synapse models
    2. Layers
    3. Connection models
    4. Individual connections
    5. Population and connection recorders
3. __Create the network__ in NEST
4. __Access the network elements__ (GIDs, etc)
4. Export and reuse the parameter tree allowing us to __replicate the network__

## Initialize an empty network

When initialized without argument or with an empty tree as an argument, all the expected subtrees are initialized as empty.

When building the elements interactively, the network's parameters are updated

In [ ]:
net = Network()

In [ ]:
# Network parameter tree is empty:
net.tree

## Build the network components (models, layers, connections, recorders)


### Define new models

We can define neuron, recorder, stimulator and synapse models with arbitrary parameters from parameter trees.

Each leaf corresponds to a new (named) model. Its ``nest_params`` and ``params`` are hierarchically inherited.

The ``nest_model`` used is specified in the leaf's ``params``

#### ``'neuron_models'`` tree:

Initalize ``Network.neuron_models`` with the ``Network.build_neuron_models`` method

In [ ]:
neuron_models_tree = ParamsTree.read(PARAMS_DIR/'models.yml').children['neuron_models']
pprint(neuron_models_tree)

In [ ]:
net.build_neuron_models(neuron_models_tree)

In [ ]:
# The neuron models are saved as an attribute for the Network object
print("\n``Network.neuron_models`` :")
pprint(net.neuron_models)

In [ ]:
pprint(net.neuron_models['l1_exc'])
pprint(net.neuron_models['l1_exc'].params)
pprint(net.neuron_models['l1_exc'].nest_params)

#### ``'recorder_models'`` tree:

Initalize ``Network.recorder_models`` with the ``Network.build_recorder_models`` method

-> Same thing as for neuron models

In [ ]:
# ``Network.build_*`` methods accept as argument ``ParamsTree`` objects, but also tree-like dictionaries 
recorder_models_tree = ParamsTree.read(PARAMS_DIR/'models.yml').children['recorder_models']
recorder_models_tree

In [ ]:
# ``Network.build_*`` methods accept as argument ``ParamsTree`` objects, but also tree-like dictionaries 
recorder_models_tree = recorder_models_tree.asdict()
recorder_models_tree

In [ ]:
net.build_recorder_models(recorder_models_tree)

In [ ]:
print("\n``Network.recorder_models`` :")
pprint(net.recorder_models)

#### ``'synapse_model'`` tree:

Initalize ``Network.synapse_models`` with the ``Network.build_synapse_model`` method

-> Same thing as for neuron models, with as a bonus a convenient way of specifying the receptor type of the synapse

-> If specifying the ``receptor_type`` and ``target_model`` in the ``SynapseModel`` params, the corresponding port is determined automatically

In [ ]:
synapse_models_tree = ParamsTree.read(PARAMS_DIR/'models.yml').children['synapse_models']
synapse_models_tree

In [ ]:
net.build_synapse_models(synapse_models_tree)

In [ ]:
print("\n``Network.synapse_models`` :")
pprint(net.synapse_models)

Note that the ``receptor_type`` nest_parameter was inferred

### Define layers

As for models, we can create ``nest.Topology`` layers from the leaves of a tree.

The elements can be nest models with their default parameters, or the ones we just created with custom params

For layers of stimulator devices, we can use the ``InputLayer`` object, which can automatically create paired parrot neurons for each stimulator units, by adding ``type: 'InputLayer'``
to the params


#### ``'layers'`` tree:


In [ ]:
layer_tree = ParamsTree.read(PARAMS_DIR/'layers.yml')
layer_tree

In [ ]:
net.build_layers(layer_tree)

In [ ]:
pprint(net.layers)

In [ ]:
print("'l1' layer")
pprint(net.layers['l1'].params)
pprint(net.layers['l1'].nest_params)

### Define connections

We create connections using a two step process:

1. Create ``ConnectionModel`` objects from a tree. Each named leaf will define a template from which individual connections can inherit their parameters
2. Create ``Connection`` objects from a list, specifying for each item the source layer x population, target layer x population and the connection model to inherit parameters from

#### 1- Define templates from the `connection_models` tree



In [ ]:
conn_model_tree = ParamsTree.read(PARAMS_DIR/'connections.yml').children['connection_models']
conn_model_tree

In [ ]:
net.build_connection_models(conn_model_tree)

In [ ]:
net.connection_models

### 2- Define individual connections from the `topology` tree

The list of connections is defined in the `connections` params of the `topology` tree

Check out the doc of `Network.build_connections` for expected formatting

In [ ]:
conns_tree = ParamsTree.read(PARAMS_DIR/'connections.yml').children['topology']
conns_tree

In [ ]:
net.connection_models

In [ ]:
net.build_connections(conns_tree)

In [ ]:
net.connections

## Define recorders from the `recorders` tree

Similarly to the `topology` tree, recorders are defined from lists.


We separate recorders connected to synapses (eg weight recorder) and those connected to units (eg spike detectors),
which are defined in the `connection_recorders` and `population_recorders` params (resp.) of the `recorders` tree.


Check out the doc of the ``Network.build_recorders``, ``Network.build_population_recorders`` and
``Network.build_connection_recorders`` methods for expected formatting

The parameters of the recorders can be changed by using custom recorder models (in the `recorder_models` tree, see above) 

In [ ]:
recorders_tree = ParamsTree.read(PARAMS_DIR/'recorders.yml').children['recorders']
recorders_tree

In [ ]:
net.build_recorders(recorders_tree)

In [ ]:
net.population_recorders

In [ ]:
net.connection_recorders

## Create the network

In [ ]:
nest.ResetKernel()

In [ ]:
net.create()

## Examine the network

nets provides convenient ways of accessing the objects in nest

### Check the defaults of the created models

In [ ]:
print("`l1_exc` neuron models `nest_params`: ", net.neuron_models['l1_exc'].nest_params)

In [ ]:
print('Corresponding params of the `l1_exc` model in nest:', nest.GetDefaults('l1_exc', list(net.neuron_models['l1_exc'].nest_params.keys())))

### Access the layers' units

In [ ]:
net.layers['l1'].gids(location=(0, 0), population='l1_exc')

### Access the connections created in NEST

In [ ]:
conn = net.connections[0]
conn

In [ ]:
nest_conns = nest.GetConnections(
    source=conn.source.gids(conn.source_population),
    target=conn.target.gids(conn.target_population),
    synapse_model=conn.nest_params['synapse_model']
)
nest_conns[0:5]

### Access the recorders

In [ ]:
rec = net.population_recorders[0]
print(rec, rec.gid, rec.model, rec.layer, rec.population_name)

In [ ]:
connrec = net.connection_recorders[0]
print(connrec, connrec.gid, connrec.model)

## Save and recreate the network

When building each of the network's elements using the `Network.build_*` methods, the `Network.tree` ParamsTree was updated

In [ ]:
net.tree

#### We can save the parameter tree defining the whole network...

In [ ]:
net.tree.write(PARAMS_DIR/'network_tree.yml')

#### And use it to recreate another identical network

In [ ]:
net2 = Network(ParamsTree.read(PARAMS_DIR/'network_tree.yml'))

# Change the network's state

# Run a single session

In [ ]:
session_params = {
    'simulation_time': 1000.0,
    'reset_network': False,
    'record': True,
    'unit_changes': [
        {
            'layers': ['input_layer'],
            'population': 'poisson_generator',
            'proportion': 1.0,
            'params': {
                'rate': 100.0
            }
        }
    ]
}

In [ ]:
session_1 = Session(
    'session_1',
    session_params,
)

In [ ]:
session_1.run(net)

In [ ]:
session_1.duration

# Let's familiarize ourselves with the ``Simulation`` object


## Initialize an empty Simulation object

Empty network, no kernel initialization, etc...

In [5]:
sim = Simulation(output_dir=OUTPUT_DIR)

2020-05-28 17:22:35,352 [nets.utils.validation] INFO: 'None' tree: adding empty child kernel
2020-05-28 17:22:35,358 [nets.utils.validation] INFO: 'None' tree: adding empty child simulation
2020-05-28 17:22:35,361 [nets.utils.validation] INFO: 'None' tree: adding empty child session_models
2020-05-28 17:22:35,362 [nets.utils.validation] INFO: 'None' tree: adding empty child network
2020-05-28 17:22:35,366 [nets.utils.validation] INFO: Object `simulation`: params: using default value for optional parameters:
{'input_dir': 'input', 'output_dir': 'output', 'sessions': []}
2020-05-28 17:22:35,368 [nets.utils.validation] INFO: Object `kernel`: params: using default value for optional parameters:
{'extension_modules': [], 'nest_seed': 1, 'python_seed': 1}
2020-05-28 17:22:35,369 [nets.simulation] INFO: Initializing NEST kernel and seeds...
2020-05-28 17:22:35,370 [nets.simulation] INFO:   Resetting NEST kernel...
2020-05-28 17:22:35,377 [nets.simulation] INFO:   Setting NEST kernel status...

In [6]:
# The data and metadata is saved there. For now it's all empty
!ls {OUTPUT_DIR}

data               git_hash           parameter_tree.yml session_times.yml


## Initialize the NEST kernel

``Simulation.init_kernel??`` for doc

In [7]:
kernel_tree = {
    'params':
        {
            'nest_seed': 10,
            'extension_modules': [],
        },
    'nest_params':
        {
            'resolution': 0.5,
        },
}

In [8]:
sim.init_kernel(kernel_tree)

2020-05-28 17:22:50,630 [nets.utils.validation] INFO: Object `kernel`: params: using default value for optional parameters:
{'python_seed': 1}
2020-05-28 17:22:50,631 [nets.simulation] INFO: Initializing NEST kernel and seeds...
2020-05-28 17:22:50,632 [nets.simulation] INFO:   Resetting NEST kernel...
2020-05-28 17:22:50,641 [nets.simulation] INFO:   Setting NEST kernel status...
2020-05-28 17:22:50,644 [nets.simulation] INFO:     Calling `nest.SetKernelStatus({'resolution': 0.5})`
2020-05-28 17:22:50,645 [nets.simulation] INFO:     Calling `nest.SetKernelStatus({'data_path': 'output/data', 'grng_seed': 11, 'rng_seeds': range(12, 13)})
2020-05-28 17:22:50,647 [nets.simulation] INFO:   Finished setting NEST kernel status
2020-05-28 17:22:50,648 [nets.simulation] INFO:   Installing external modules...
2020-05-28 17:22:50,648 [nets.simulation] INFO:   Finished installing external modules
2020-05-28 17:22:50,649 [nets.simulation] INFO:   Setting Python seed: 1
2020-05-28 17:22:50,650 [net

In [9]:
# nest_params have been passed to nest.SetKernelStatus
nest.GetKernelStatus('resolution')

0.5

In [12]:
# The raw data will be saved in the output directory
nest.GetKernelStatus('data_path')

'output/data'

In [10]:
# The kernel params are saved in the simulation's tree
sim.tree.children['kernel']

ParamsTree(name='kernel', parent='None')
  params:
    nest_seed: 10
    extension_modules: []
  nest_params:
    resolution: 0.5
  

## Create a network

We build and create the same network by passing the network tree,
using the ``Simulation.create_network`` method

In [13]:
net_tree = ParamsTree.read(PARAMS_DIR/'network_tree.yml')

In [14]:
sim.create_network(net_tree)

2020-05-28 17:23:28,322 [nets.simulation] INFO: Building network.
2020-05-28 17:23:28,334 [nets.network] INFO: Build N=2 ``Model`` objects
2020-05-28 17:23:28,335 [nets.network] INFO: Build N=2 ``SynapseModel`` objects
2020-05-28 17:23:28,338 [nets.network] INFO: Build N=2 ``Model`` objects
2020-05-28 17:23:28,340 [nets.network] INFO: Build N=2 ``Layer`` or ``InputLayer`` objects.
2020-05-28 17:23:28,341 [nets.utils.validation] INFO: Object `conn_1_AMPA`: params: using default value for optional parameters:
{'type': 'topological'}
2020-05-28 17:23:28,342 [nets.utils.validation] INFO: Object `conn_2_GABAA`: params: using default value for optional parameters:
{'type': 'topological'}
2020-05-28 17:23:28,343 [nets.network] INFO: Build N=2 ``ConnectionModel`` objects
2020-05-28 17:23:28,348 [nets.network] INFO: Build N=3 ``TopoConnection`` objects
2020-05-28 17:23:28,352 [nets.network] INFO: Build N=1 population recorders.
2020-05-28 17:23:28,354 [nets.network] INFO: Build N=1 connection r

In [15]:
# Network object was created and can be accessed as we learnt
sim.network.layers['l1'].gid

(52,)

In [16]:
# network tree was saved and can be re-used
sim.tree.children['network']

ParamsTree(name='network', parent='None')
  params: {}
  nest_params: {}
  neuron_models:
    params: {}
    nest_params: {}
    my_neuron:
      params:
        nest_model: ht_neuron
      nest_params:
        g_KL: 1.0
        g_NaL: 1.0
      l1_exc:
        params: {}
        nest_params:
          V_m: -44.0
  
  ... [107 lines] ...

        - l1
        populations:
        - l1_exc
        model: my_multimeter
      connection_recorders:
      - source_layers:
        - l1
        source_population: l1_exc
        target_layers:
        - l1
        target_population: l1_inh
        connection_model: conn_1_AMPA
        model: weight_recorder
    nest_params: {}
  

## Create some sessions

### so we can run the network in specific conditions

### Build session models from a tree

In [17]:
# Notice the hierarchical inheritance as before
session_models_tree = ParamsTree.read(PARAMS_DIR/'session_models.yml').children['session_models']
print(session_models_tree)

params:
  record: true
  simulation_time: 100.0
nest_params: {}
warmup:
  params:
    record: false
  nest_params: {}
high_input:
  params:
    unit_changes:
    - layers:
      - input_layer
      population: poisson_generator
      params:
        rate: 100.0
  nest_params: {}
low_input:
  params:
    unit_changes:
    - layers:
      - input_layer
      population: poisson_generator
      params:
        rate: 50.0
  nest_params: {}



In [18]:
sim.build_session_models(session_models_tree)

2020-05-28 17:23:35,793 [nets.simulation] INFO: Build N=3 session models


In [19]:
sim.session_models

{'warmup': ParamsTree(name='warmup', parent='session_models')
   params:
     record: false
   nest_params: {}
   ,
 'high_input': ParamsTree(name='high_input', parent='session_models')
   params:
     unit_changes:
     - layers:
       - input_layer
       population: poisson_generator
       params:
         rate: 100.0
   nest_params: {}
   ,
 'low_input': ParamsTree(name='low_input', parent='session_models')
   params:
     unit_changes:
     - layers:
       - input_layer
       population: poisson_generator
       params:
         rate: 50.0
   nest_params: {}
   }

### Build a list of sessions from the ``session_models`` templates

In [20]:
sessions_order = ['warmup', 'high_input', 'low_input', 'high_input', 'low_input']

In [21]:
sim.build_sessions(sessions_order)

2020-05-28 17:23:38,184 [nets.simulation] INFO: Build N=5 sessions
2020-05-28 17:23:38,188 [nets.session] INFO: Creating session "00_warmup"
2020-05-28 17:23:38,189 [nets.utils.validation] INFO: Object `00_warmup`: params: using default value for optional parameters:
{'inputs': {},
 'reset_network': False,
 'synapse_changes': [],
 'unit_changes': []}
2020-05-28 17:23:38,191 [nets.session] INFO: Creating session "01_high_input"
2020-05-28 17:23:38,193 [nets.utils.validation] INFO: Object `01_high_input`: params: using default value for optional parameters:
{'inputs': {}, 'reset_network': False, 'synapse_changes': []}
2020-05-28 17:23:38,193 [nets.session] INFO: Creating session "02_low_input"
2020-05-28 17:23:38,196 [nets.utils.validation] INFO: Object `02_low_input`: params: using default value for optional parameters:
{'inputs': {}, 'reset_network': False, 'synapse_changes': []}
2020-05-28 17:23:38,198 [nets.session] INFO: Creating session "03_high_input"
2020-05-28 17:23:38,199 [nets

In [22]:
# Notice the session names
sim.sessions

[Session(00_warmup, {'inputs': {},
  'record': False,
  'reset_network': False,
  'simulation_time': 100.0,
  'synapse_changes': [],
  'unit_changes': []}),
 Session(01_high_input, {'inputs': {},
  'record': True,
  'reset_network': False,
  'simulation_time': 100.0,
  'synapse_changes': [],
  'unit_changes': [{'layers': ['input_layer'],
                    'params': {'rate': 100.0},
                    'population': 'poisson_generator'}]}),
 Session(02_low_input, {'inputs': {},
  'record': True,
  'reset_network': False,
  'simulation_time': 100.0,
  'synapse_changes': [],
  'unit_changes': [{'layers': ['input_layer'],
                    'params': {'rate': 50.0},
                    'population': 'poisson_generator'}]}),
 Session(03_high_input, {'inputs': {},
  'record': True,
  'reset_network': False,
  'simulation_time': 100.0,
  'synapse_changes': [],
  'unit_changes': [{'layers': ['input_layer'],
                    'params': {'rate': 100.0},
                    'population': 'po

### Run all the sessions

In [23]:
print('kernel time: ', nest.GetKernelStatus('time'))

kernel time:  0.0


In [24]:
sim.run()

2020-05-28 17:23:43,384 [nets.simulation] INFO: Running 5 sessions...
2020-05-28 17:23:43,384 [nets.simulation] INFO: Running session: '00_warmup'...
2020-05-28 17:23:43,386 [nets.session] INFO: Initializing session...
-> Changing synapses's state.: 0it [00:00, ?it/s]
2020-05-28 17:23:43,393 [nets.network.recorders] INFO:   Setting status for recorder my_multimeter_l1_l1_exc: {'start': 100.0}
2020-05-28 17:23:43,394 [nets.network.recorders] INFO:   Setting status for recorder weight_recorder_conn_1_AMPA-l1-l1_exc-l1-l1_inh: {'start': 100.0}
2020-05-28 17:23:43,397 [nets.session] INFO: Finished initializing session

2020-05-28 17:23:43,399 [nets.session] INFO: Running session '00_warmup' for 100 ms
2020-05-28 17:23:43,554 [nets.session] INFO: Finished running session
2020-05-28 17:23:43,557 [nets.session] INFO: Session '00_warmup' virtual running time: 100 ms
2020-05-28 17:23:43,559 [nets.session] INFO: Session '00_warmup' real running time: 0h:00m:00s
2020-05-28 17:23:43,560 [nets.simu

In [25]:
print('kernel time: ', nest.GetKernelStatus('time'))

kernel time:  500.0


### Save the simulation metadata

In [26]:
sim.save_metadata()

2020-05-28 17:24:00,870 [nets.simulation] INFO: Saving simulation metadata...
2020-05-28 17:24:00,871 [nets.simulation] INFO: Creating output directory: output
2020-05-28 17:24:00,873 [nets.io.save] INFO: Clearing directory: output
2020-05-28 17:24:00,875 [nets.io.save] INFO: Clearing directory: output
2020-05-28 17:24:00,879 [nets.io.save] INFO: Clearing directory: output/data
2020-05-28 17:24:00,883 [nets.io.save] INFO: Clearing directory: output/data
2020-05-28 17:24:00,885 [nets.io.save] INFO: Clearing directory: output/data
2020-05-28 17:24:00,887 [nets.io.save] INFO: Clearing directory: output
2020-05-28 17:24:01,048 [nets.simulation] INFO: Finished saving simulation metadata


In [ ]:
!ls {OUTPUT_DIR}

In [ ]:
!cat {OUTPUT_DIR/'session_times.yml'}

In [ ]:
!ls {OUTPUT_DIR/'data'}

## Load simulation outputs

In [ ]:
import nets.io.load
from pathlib import Path

OUTPUT_DIR = Path(sim.output_dir)

In [ ]:
# Load the start and end time for each session
session_times = nets.io.load.load_session_times(OUTPUT_DIR) 
print(session_times)  # {<session_name>: (<session_start>, <session_end>)}

In [ ]:
## Load data from a specific recorder.

recorder_metadata_path = OUTPUT_DIR/'data/multimeter_l1_l1_exc.yml'

# All relevant information about this recorder and the population it's
# connected to are contained in its metadata file
recorder_metadata = nets.io.load.load_yaml(recorder_metadata_path)
print(f'Metadata keys: {recorder_metadata.keys()}')

In [ ]:
# We can load the raw data as pandas dataframe
df = nets.io.load.load(recorder_metadata_path)
print(df[0:5])

In [ ]:
all_recorder_metadata_paths = nets.io.load.metadata_paths(OUTPUT_DIR)
print(all_recorder_metadata_paths)

In [ ]:
for metadata_path in all_recorder_metadata_paths:
  print(f'Recorder: {metadata_path.name}')
  print(f'{nets.io.load.load(metadata_path)[0:5]}\n')

## Replicate the simulation

In [ ]:
params = ParamsTree.read(OUTPUT_DIR/'parameter_tree.yml')

In [ ]:
# sim = nets.Simulation(params)
# sim.run()